# Summer of Bitcoin Coding Challenge
The problem has been broken down into 4 major steps which is followed by an observation regarding the algorithm used at the end.

## Step1
Read the file into a dataframe using pandas module and store the minimum weight in a variable `min_wt`

In [246]:
import pandas as pd
data  = pd.read_csv("mempool.csv")
data['parents'] = data['parents ']
data.drop(['parents '], axis = 1)
min_wt = data['weight'].min()

## Step 2
Add a new parameter in dataframe which is equal to fee and weight ratio and sort it accordingly. The reason behind taking fee/weight ratio is that we want to maximise fee keeping wieght minimum, therefore we will sort the dataframe in descending order of fee/weight ratio

In [247]:
data["fee/weight"] = data["fee"] / data["weight"]
data = data.sort_values('fee/weight', ascending = False)
data.reset_index(inplace=True)
data = data.drop(['index'], axis = 1)

## Step 3
Now we have a sorted dataframe, the first element has maximum fee to weight ratio. Now one by one we will check that if the transaction is valid or not in from top to bottom order (i.e that its parents are already in block if they exists and its weight is less than the difference between current block weight and the maximum block weight). If it is a valid transaction then we will delete it from the data frame and add it to a list called `blockdata` and deleted from the dataframe and if it is not valid then we will first check that the difference (between current block weight and maximum block weight) is greater than the `min_wt`, if it is true then we will move to the next transaction and check for it and if not then we break the loop.

In [248]:
blockdata = list()
total_fee = 0
total_wt = 0
max_wt = 4000000
i = 0

The folowing function checks that whether the parents of the given transaction are present in the block or not.

In [249]:
def check_trn(index):
    parents = str(data['parents'][index])
    if (parents == 'nan'):
        return True
    data['parents'][index].split(';')
    for parent in parents:
        if parent in blockdata:
            return False
    return True

In [250]:
def check_wt(index):
    wt = data['weight'][index]
    if max_wt - total_wt > wt:
        return True
    return False

In [251]:
len(data.axes[0])

5214

In [252]:
while(i < len(data.axes[0])):
    if check_wt(i) and check_trn(i):
        blockdata.append(str(data['tx_id'][i]))
        total_fee += data['fee'][i]
        total_wt += data['weight'][i]
        data = data.drop(i)
        data = data.reset_index(drop=True)
        i = 0
    elif max_wt - total_wt < min_wt:
        break
    else:
        i += 1

In [253]:
total_wt

3999784

In [254]:
total_fee

5817973

## Step 4
Now `blockdata` contains all the trasactions. we will create a file named `block.txt` and store the data inside it.

In [255]:
block = open('block.txt', 'w')
for trn in blockdata:
    block.write(trn + '\n')
block.close

<function TextIOWrapper.close()>

# An Observation
The Algorithm that I used is not the optimum solution of the problem, there are many such cases in which it won't maximise the fee one of the case is as follows:  
Let the `tx_id` be `x`, `y` and `z` with following fees and weights, you are given that the maximum weight is 30 units and none of the transactions have a parent transaction 

In [256]:
test = pd.DataFrame([['x', 15, 10, 1.5], ['y', 20, 20, 1], ['z', 30, 30, 1]], columns=['tx_id', 'fee', 'weight', 'fee/weight'])
test

,tx_id,fee,weight,fee/weight
0,x,15,10,1.5
1,y,20,20,1.0
2,z,30,30,1.0


According to my algorithm the block would only contain `x` as it has maximum `fee/weight` ratio but it only provides with 15 units of fees while on the other hand `y` and `z` both are better choice for maximising the `fee` as compared to `x`. I could not come up with an algorithm that accounts for these cases and provide a guaranteed maximum fee.